In [40]:
import findspark
findspark.init()

In [41]:
import pyspark
sc = pyspark.SparkContext(appName='myapp')

## Accumulators and broadcasts

In [42]:
inputFile = './data/README.md'
outputDir = './data/'

In [43]:
file = sc.textFile(inputFile)
blankLines = sc.accumulator(0) # Create Accumulator[Int] initialized to 0

def extractCallSigns(line):
    global blankLines   # Make the global variable accessible, otherwise get "reference before assignment error"
    if (line == ""):
        blankLines += 1
    return line.split(" ")

callSigns = file.flatMap(extractCallSigns)
callSigns.saveAsTextFile(outputDir + "/callsigns")
print("Blank lines: %d" % blankLines.value)

Py4JJavaError: An error occurred while calling o183.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/allenwang/myDoc/651 - Adv. Computing/learning spark/data/callsigns already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sca

**Caution!**

> How does this interact with accumulators? The end result is that for accumulators used in actions, Spark applies each task’s update to each accumulator only once. Thus, if we want a reliable absolute value counter, regardless of failures or multiple evaluations, we **must** put it inside an action like foreach().

**Also, since version 2.x we have AccumulatorV2, so ...**

In [ ]:
# Look up the locations of the call signs on the
# RDD contactCounts. We load a list of call sign
# prefixes to country code to support this lookup.
signPrefixes = sc.broadcast(loadCallSignTable())

def processSignCount(sign_count, signPrefixes):
    country = lookupCountry(sign_count[0], signPrefixes.value)
    count = sign_count[1]
    return (country, count)

countryContactCounts = (contactCounts
                       .map(processSignCount)
                       .reduceByKey((lambda x, y: x+ y)))

countryContactCounts.saveAsTextFile(outputDir + "/countries.txt")

**Broadcast variable looks the same as in native Python, but it is actually implemented so that it is only sent to each node only once in an efficient way.**

## Per-partition operations
**This allows us to do setup stuff only once for each partition, e.g. creating DB connection or initializing a random number generator.**

## Sourcing other programs
**In Spark, pipe() provides a way of using scripts in other languages, as long as the sourced script can read from and write to  Unix standard streams.**
**e.g. in R, if the script read from stdin and writes to stdout, then we can pipe this R script into our analysis pipeline in Spark.**

---

## Deploying Spark program on clusters

The spark-submit script let's us deploy programs locally or on a cluster specified by various parameters.

```shell
bin/spark-submit my_script.py
bin/spark-submit --master spark://host:7707 --executor-memory 10g my_script.py
```

## Tuning and debugging Spark programs

**Configuring a SparkContext**

In [46]:
# Construct a SparkConf object
conf = pyspark.SparkConf()
conf.set("spark.app.name", "My Spark App")
conf.set("spark.master", "local[4]")
conf.set("spark.ui.port", "36000") # Override the default port

In [ ]:
# Create a SparkContext with this configuration
sc = SparkContext(conf)

**Analysing Performance:**

- There is a web UI for analyzing jobs at localhost:4040
- Coding faster code
    - 

---

In [39]:
sc.stop()